In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [3]:
data = pd.read_csv('./data/fake_job_postings.csv')
data.head(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [5]:
new_df = data[["title", "location", "company_profile", "description", "requirements", 
               "benefits", "telecommuting","has_company_logo","has_questions","employment_type",
               "required_experience","industry","function","fraudulent"]]
#new_df = new_df.dropna()
#new_df.info()

# Preprocessing

In [1]:
#pip install nltk

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# text preprocessing function
def preprocess_text(text):

    # tokenization
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package punkt to /Users/lilyw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lilyw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lilyw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/lilyw/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
# replace NA
text_df = new_df[["title", "company_profile", "description", "requirements", "benefits"]]
text_df['title'].fillna('unknown', inplace=True)
text_df['company_profile'].fillna('unknown', inplace=True)
text_df['description'].fillna('unknown', inplace=True)
text_df['requirements'].fillna('unknown', inplace=True)
text_df['benefits'].fillna('unknown', inplace=True)

# process text columns
text_df['preprocessed_title'] = text_df['title'].apply(preprocess_text)
text_df['preprocessed_company_profile'] = text_df['company_profile'].apply(preprocess_text)
text_df['preprocessed_description'] = text_df['description'].apply(preprocess_text)
text_df['preprocessed_requirements'] = text_df['requirements'].apply(preprocess_text)
text_df['preprocessed_benefits'] = text_df['benefits'].apply(preprocess_text)

text_df.head(5)

/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_29284/523841265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['title'].fillna('unknown', inplace=True)
/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_29284/523841265.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['company_profile'].fillna('unknown', inplace=True)
/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_29284/523841265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

,title,company_profile,description,requirements,benefits,preprocessed_title,preprocessed_company_profile,preprocessed_description,preprocessed_requirements,preprocessed_benefits
0,Marketing Intern,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,marketing intern,'re food52 've created groundbreaking award-wi...,food52 fast-growing james beard award-winning ...,experience content management system major plu...,unknown
1,Customer Service - Cloud Video Production,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,customer service cloud video production,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get usthrough part 90 second team gain experie...
2,Commissioning Machinery Assistant (CMA),Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,commissioning machinery assistant cma,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre-commissioning commissioning proc...,unknown
3,Account Executive - Washington DC,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,account executive washington dc,passion improving quality life geography heart...,company esri – environmental system research i...,education bachelor ’ master ’ gi business admi...,culture anything corporate—we collaborative cr...
4,Bill Review Manager,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,bill review manager,spotsource solution llc global human capital m...,job title itemization review managerlocation f...,qualification rn license state texasdiploma ba...,full benefit offered


In [53]:
text_df.shape

(17880, 10)

# Text Vectorization

In [54]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

columns_to_vectorize = ["preprocessed_title", "preprocessed_company_profile", "preprocessed_description", "preprocessed_requirements", "preprocessed_benefits"]

tfidf_data = {}

tfidf_vectorizer = TfidfVectorizer(max_features=100)

for column in columns_to_vectorize:

    tfidf_matrix = tfidf_vectorizer.fit_transform(text_df[column])
    #print(type(tfidf_matrix))
    #print(tfidf_matrix)
    #print(tfidf_matrix.shape)

    tfidf_data[column] = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


print(tfidf_data)

#tfidf_df = pd.DataFrame(tfidf_data)
#print(tfidf_data.head())
#print(tfidf_data.shape)


{'preprocessed_title':         16   18  abroad   account  admin  administrative  administrator  \
0      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
1      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
2      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
3      0.0  0.0     0.0  0.700986    0.0             0.0            0.0   
4      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
...    ...  ...     ...       ...    ...             ...            ...   
17875  0.0  0.0     0.0  0.698317    0.0             0.0            0.0   
17876  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17877  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17878  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17879  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   

       agent  analyst  android  ...  system  teacher  team  technical  \
0  

In [61]:
# combine dataframe
vect_df = pd.concat([tfidf_data["preprocessed_title"], tfidf_data["preprocessed_company_profile"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_description"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_requirements"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_benefits"]], axis=1)
#vect_df = pd.concat([vect_df, new_df["fraudulent"]], axis=1)

y = new_df["fraudulent"]
vect_df.shape

(17880, 500)

# Split data & Model Training

In [66]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vect_df, y, test_size=0.2, random_state=42)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

14304
3576
14304
3576


In [70]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)


MultinomialNB()

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(nb_clf.predict(X_test), y_test)


0.9460290827740492

In [74]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train, y_train)


LinearSVC()

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(svc.predict(X_test), y_test)

0.9720357941834452